In [1]:
import pandas as pd
import numpy as np
import os
os.getcwd() 

'/home/balleng/Dropbox/Gustavo/papers/genomeMelody/divtime_christmas/00_tree_inference/code'

In [2]:
christmas = pd.read_csv("../data/netvor++_0.3_2023-03-21.csv")
christmas.head()

,id,corpus_id,incipit,cantus_id,mode,finalis,differentia,siglum,position,folio,...,office_id,source_id,melody_id,drupal_path,full_text,full_text_manuscript,volpiano,notes,dataset_name,dataset_idx
0,NaN,NaN,Constantes estote videbitis,007040a,4S,NaN,NaN,A-KN 1010,01,021r,...,office_v2,NaN,NaN,NaN,Constantes estote videbitis auxilium domini su...,NaN,1--dh-hhe-gfe--df-efgh-g--hi-h-gf-gh--gg-dfefg...,NaN,NaN,NaN
1,NaN,NaN,Bethleem non es minima in principibus,001737,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,010v,...,office_p,NaN,NaN,NaN,Bethleem non es minima in principibus,NaN,1---cd--ef-e-d--d---gf---ghg---fe--de--f-e-d-e...,NaN,NaN,NaN
2,NaN,NaN,Bethleem non es minima in principibus,001737,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,070v,...,office_v,NaN,NaN,NaN,Bethleem non es minima in principibus,NaN,1---cd--ef-e-d--d---gf---ghg---7fe--de--f-e-de...,NaN,NaN,NaN
3,NaN,NaN,Constantes estote videbitis auxilium domini su...,007040a,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,071r,...,office_v,NaN,NaN,NaN,Constantes estote videbitis auxilium domini su...,NaN,1---dg--7h--g-f-e---df--egh--g---hk--h--gf--gh...,NaN,NaN,NaN
4,NaN,NaN,Cum esset desponsata mater Ihesu Maria,002000,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,071v,...,office_v,NaN,NaN,NaN,Cum esset desponsata mater Ihesu Maria,NaN,1---g---7g-f-e-d--e-fg---g--g--hg--f---g--h---...,NaN,NaN,NaN


In [3]:
# numeric symbols encoding structure of the staff should be removed from the volpiano string before translation 
# and alignment
# some g-clefs are encoded 1-- rather than 1---
christmas.volpiano = christmas.volpiano.str.replace('1---', '')
christmas.volpiano = christmas.volpiano.str.replace('1--', '')
christmas.volpiano = christmas.volpiano.str.replace('---3', '')
christmas.volpiano = christmas.volpiano.str.replace('---4', '')
christmas.volpiano = christmas.volpiano.str.replace('4', '') # this appears as an erroneous instance of the previous code
christmas.volpiano = christmas.volpiano.str.replace('---5', '') # this one is not documented but appears in some chants at the end
christmas.volpiano = christmas.volpiano.str.replace('777', '')
christmas.volpiano = christmas.volpiano.str.replace('77', '')
christmas.volpiano = christmas.volpiano.str.replace('7', '')
christmas.volpiano = christmas.volpiano.str.replace('6------6', '')
christmas.volpiano = christmas.volpiano.str.replace('---6', '')
christmas.volpiano = christmas.volpiano.str.replace('-----6', '')
christmas.volpiano = christmas.volpiano.str.replace('------6', '')
christmas.volpiano = christmas.volpiano.str.replace('-------6', '')
christmas.volpiano = christmas.volpiano.str.replace('--------6', '')
christmas.volpiano = christmas.volpiano.str.replace('---------6', '')


In [4]:
# deep copy to preserve the original information
christmas_translated = christmas.copy(deep=True)
# translate the --- to ~ and -- to |
christmas_translated.volpiano = christmas_translated.volpiano.str.replace('---', '~')
christmas_translated.volpiano = christmas_translated.volpiano.str.replace('--', '|')
# check that translation is not affecting the original object
christmas.volpiano.head()

0    dh-hhe-gfe--df-efgh-g--hi-h-gf-gh--gg-dfefgh-d...
1    cd--ef-e-d--d---gf---ghg---fe--de--f-e-d-e---c...
2    cd--ef-e-d--d---gf---ghg---fe--de--f-e-de---c-...
3    dg--h--g-f-e---df--egh--g---hk--h--gf--gh---g-...
4    g---g-f-e-d--e-fg---g--g--hg--f---g--h---kj--h...
Name: volpiano, dtype: object

In [5]:
christmas_translated.volpiano.head()

0    dh-hhe-gfe|df-efgh-g|hi-h-gf-gh|gg-dfefgh-def|...
1    cd|ef-e-d|d~gf~ghg~fe|de|f-e-d-e~c~e|g|f|ghg~e...
2    cd|ef-e-d|d~gf~ghg~fe|de|f-e-de~c~e|g|f|ghg~ed...
3    dg|h|g-f-e~df|egh|g~hk|h|gf|gh~g|df|egh|df~efg...
4    g~g-f-e-d|e-fg~g|g|hg|f~g|h~kj|hg~h|h|hg~fg|g~...
Name: volpiano, dtype: object

In [6]:
# create a new column with the combination of cantus id and the incipit

# first check that either cantus_id or corpus_id has only uniques
christmas.cantus_id.value_counts()

003511           21
004195           21
007040a          17
605019           16
001737           16
002000           15
201896           10
007040b           9
001546            6
008408a           4
007040.Tp1        3
004834            2
g01598:cs         1
006882            1
006882a           1
002642            1
006882Pzc         1
004657            1
003670            1
003511:cs         1
605019:cs         1
004195:cs         1
001546:cs         1
007040.Tp1:cs     1
008408a:cs        1
002000:cs         1
g01564.Tp4:cs     1
007040b:cs        1
002922:cs         1
001737:cs         1
003608            1
001084            1
g01598            1
Name: cantus_id, dtype: int64

In [7]:
# use the cantus id to pick comparable melodies. Remove the :cz particle as this is just text in 
# czech of essentially the same melody
christmas.loc[:,'cantus_id_clean'] = christmas.loc[:,'cantus_id'].apply(lambda row: row.replace(":cs", ""))
christmas.cantus_id_clean.value_counts()

003511        22
004195        22
007040a       17
605019        17
001737        17
002000        16
201896        10
007040b       10
001546         7
008408a        5
007040.Tp1     4
g01598         2
004834         2
003608         1
002922         1
g01564.Tp4     1
001084         1
003670         1
004657         1
002642         1
006882a        1
006882         1
006882Pzc      1
Name: cantus_id_clean, dtype: int64

In [8]:
# repeat it for the translated version
christmas_translated.loc[:,'cantus_id_clean'] = christmas_translated.loc[:,'cantus_id'].apply(lambda row: row.replace(":cs", ""))
christmas_translated.cantus_id_clean.value_counts()

003511        22
004195        22
007040a       17
605019        17
001737        17
002000        16
201896        10
007040b       10
001546         7
008408a        5
007040.Tp1     4
g01598         2
004834         2
003608         1
002922         1
g01564.Tp4     1
001084         1
003670         1
004657         1
002642         1
006882a        1
006882         1
006882Pzc      1
Name: cantus_id_clean, dtype: int64

In [9]:
# make just another column using the unique_id, then newline, and then the volpiano string
christmas.loc[:, 'idx'] = list(christmas.index)
christmas.loc[:,'cantusidclean_idx'] = christmas[['cantus_id_clean','idx']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
christmas.loc[:,'fasta'] = christmas[['cantusidclean_idx','volpiano']].apply(lambda row: '\n'.join(row.values.astype(str)), axis=1)
christmas.head()

,id,corpus_id,incipit,cantus_id,mode,finalis,differentia,siglum,position,folio,...,full_text,full_text_manuscript,volpiano,notes,dataset_name,dataset_idx,cantus_id_clean,idx,cantusidclean_idx,fasta
0,NaN,NaN,Constantes estote videbitis,007040a,4S,NaN,NaN,A-KN 1010,01,021r,...,Constantes estote videbitis auxilium domini su...,NaN,dh-hhe-gfe--df-efgh-g--hi-h-gf-gh--gg-dfefgh-d...,NaN,NaN,NaN,007040a,0,007040a 0,007040a 0\ndh-hhe-gfe--df-efgh-g--hi-h-gf-gh--...
1,NaN,NaN,Bethleem non es minima in principibus,001737,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,010v,...,Bethleem non es minima in principibus,NaN,cd--ef-e-d--d---gf---ghg---fe--de--f-e-d-e---c...,NaN,NaN,NaN,001737,1,001737 1,001737 1\ncd--ef-e-d--d---gf---ghg---fe--de--f...
2,NaN,NaN,Bethleem non es minima in principibus,001737,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,070v,...,Bethleem non es minima in principibus,NaN,cd--ef-e-d--d---gf---ghg---fe--de--f-e-de---c-...,NaN,NaN,NaN,001737,2,001737 2,001737 2\ncd--ef-e-d--d---gf---ghg---fe--de--f...
3,NaN,NaN,Constantes estote videbitis auxilium domini su...,007040a,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,071r,...,Constantes estote videbitis auxilium domini su...,NaN,dg--h--g-f-e---df--egh--g---hk--h--gf--gh---g-...,NaN,NaN,NaN,007040a,3,007040a 3,007040a 3\ndg--h--g-f-e---df--egh--g---hk--h--...
4,NaN,NaN,Cum esset desponsata mater Ihesu Maria,002000,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,071v,...,Cum esset desponsata mater Ihesu Maria,NaN,g---g-f-e-d--e-fg---g--g--hg--f---g--h---kj--h...,NaN,NaN,NaN,002000,4,002000 4,002000 4\ng---g-f-e-d--e-fg---g--g--hg--f---g-...


In [10]:
# make just another column using the unique_id, then newline, and then the volpiano string
christmas_translated.loc[:, 'idx'] = list(christmas_translated.index)
christmas_translated.loc[:,'cantusidclean_idx'] = christmas_translated[['cantus_id_clean','idx']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
christmas_translated.loc[:,'fasta'] = christmas_translated[['cantusidclean_idx','volpiano']].apply(lambda row: '\n'.join(row.values.astype(str)), axis=1)
christmas_translated.head()

,id,corpus_id,incipit,cantus_id,mode,finalis,differentia,siglum,position,folio,...,full_text,full_text_manuscript,volpiano,notes,dataset_name,dataset_idx,cantus_id_clean,idx,cantusidclean_idx,fasta
0,NaN,NaN,Constantes estote videbitis,007040a,4S,NaN,NaN,A-KN 1010,01,021r,...,Constantes estote videbitis auxilium domini su...,NaN,dh-hhe-gfe|df-efgh-g|hi-h-gf-gh|gg-dfefgh-def|...,NaN,NaN,NaN,007040a,0,007040a 0,007040a 0\ndh-hhe-gfe|df-efgh-g|hi-h-gf-gh|gg-...
1,NaN,NaN,Bethleem non es minima in principibus,001737,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,010v,...,Bethleem non es minima in principibus,NaN,cd|ef-e-d|d~gf~ghg~fe|de|f-e-d-e~c~e|g|f|ghg~e...,NaN,NaN,NaN,001737,1,001737 1,001737 1\ncd|ef-e-d|d~gf~ghg~fe|de|f-e-d-e~c~e...
2,NaN,NaN,Bethleem non es minima in principibus,001737,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,070v,...,Bethleem non es minima in principibus,NaN,cd|ef-e-d|d~gf~ghg~fe|de|f-e-de~c~e|g|f|ghg~ed...,NaN,NaN,NaN,001737,2,001737 2,001737 2\ncd|ef-e-d|d~gf~ghg~fe|de|f-e-de~c~e|...
3,NaN,NaN,Constantes estote videbitis auxilium domini su...,007040a,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,071r,...,Constantes estote videbitis auxilium domini su...,NaN,dg|h|g-f-e~df|egh|g~hk|h|gf|gh~g|df|egh|df~efg...,NaN,NaN,NaN,007040a,3,007040a 3,007040a 3\ndg|h|g-f-e~df|egh|g~hk|h|gf|gh~g|df...
4,NaN,NaN,Cum esset desponsata mater Ihesu Maria,002000,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,071v,...,Cum esset desponsata mater Ihesu Maria,NaN,g~g-f-e-d|e-fg~g|g|hg|f~g|h~kj|hg~h|h|hg~fg|g~...,NaN,NaN,NaN,002000,4,002000 4,002000 4\ng~g-f-e-d|e-fg~g|g|hg|f~g|h~kj|hg~h|...


In [11]:
# subsect each dataset per cantus_id_clean
# 003511 (judaea et jerusalem), 
# 004195 (orietur sicut sol salvator), 
# 007040a (constantes estote videbitis), 
# 605019 (judaea et jerusalem II), 
# 001737 (bethelem non est minima), 
# 002000 (cum esset desponsata)

### NOTE, HERE WE NEED ONLY THE ROWS CORRESPONDING TO THE SOURCES WE HAVE X %IN% sources
judjer1 = christmas[christmas['cantus_id_clean'] == '003511']
orisic = christmas[christmas['cantus_id_clean'] == '004195']
consest = christmas[christmas['cantus_id_clean'] == '007040a']
judjer2 = christmas[christmas['cantus_id_clean'] == '605019']
bethnon = christmas[christmas['cantus_id_clean'] == '001737']
cumesset = christmas[christmas['cantus_id_clean'] == '002000']

# and their translated forms
judjer1_tr = christmas_translated[christmas_translated['cantus_id_clean'] == '003511']
orisic_tr = christmas_translated[christmas_translated['cantus_id_clean'] == '004195']
consest_tr = christmas_translated[christmas_translated['cantus_id_clean'] == '007040a']
judjer2_tr = christmas_translated[christmas_translated['cantus_id_clean'] == '605019']
bethnon_tr = christmas_translated[christmas_translated['cantus_id_clean'] == '001737']
cumesset_tr = christmas_translated[christmas_translated['cantus_id_clean'] == '002000']
cumesset_tr.head()

,id,corpus_id,incipit,cantus_id,mode,finalis,differentia,siglum,position,folio,...,full_text,full_text_manuscript,volpiano,notes,dataset_name,dataset_idx,cantus_id_clean,idx,cantusidclean_idx,fasta
4,NaN,NaN,Cum esset desponsata mater Ihesu Maria,002000,NaN,NaN,NaN,A-VOR Cod. 259/I,NaN,071v,...,Cum esset desponsata mater Ihesu Maria,NaN,g~g-f-e-d|e-fg~g|g|hg|f~g|h~kj|hg~h|h|hg~fg|g~...,NaN,NaN,NaN,002000,4,002000 4,002000 4\ng~g-f-e-d|e-fg~g|g|hg|f~g|h~kj|hg~h|...
10,NaN,NaN,Cum esset desponsata mater,002000,8,NaN,NaN,A-Wn 1799**,M,017r,...,Cum esset desponsata mater Jesu Maria Joseph a...,NaN,g~ge-fd|efg~g|gh|hg|f~g|hk~kkj|hg~h|h|hg~fg|g~...,NaN,NaN,NaN,002000,10,002000 10,002000 10\ng~ge-fd|efg~g|gh|hg|f~g|hk~kkj|hg~h...
15,NaN,NaN,Cum esset desponsata mater,002000,8,NaN,NaN,CDN-Hsmu M2149.L4,M,032r,...,Cum esset desponsata mater Jesu Maria Joseph a...,NaN,g~gfefd|efg~g|gh|hhg|f~g|hk~kkj|hg~h|h|hhg~fg|...,NaN,NaN,NaN,002000,15,002000 15,002000 15\ng~gfefd|efg~g|gh|hhg|f~g|hk~kkj|hg~...
26,NaN,NaN,Cum esset desponsata mater,002000,8,NaN,NaN,CH-E 611,1M,017r,...,Cum esset desponsata mater Jesu Maria Joseph a...,NaN,g~gfd|efg~g|g|hg|f~g|h~kj|hg~h|h|hg~fg|g~g|gk|...,NaN,NaN,NaN,002000,26,002000 26,002000 26\ng~gfd|efg~g|g|hg|f~g|h~kj|hg~h|h|hg...
42,NaN,NaN,Cum esset desponsata mater Ihesu Maria,002000,NaN,NaN,NaN,CZ-HKm II A 4,NaN,027v,...,Cum esset desponsata mater Ihesu Maria,NaN,g~gg-f-e-d|e-fg~g|g|hg|f~g|h~kj|hg~h|h|hg~fg|g...,NaN,NaN,NaN,002000,42,002000 42,002000 42\ng~gg-f-e-d|e-fg~g|g|hg|f~g|h~kj|hg~...


In [12]:
christmas.volpiano.iloc[91]

'g--g--g-f-e-d---f---f--gh--h--g---g--g--hk---g--------------------------------'

In [13]:
pd.set_option('display.max_columns', None)
pd.crosstab(index=christmas['cantus_id'], columns=christmas['siglum'])

siglum,A-KN 1010,A-VOR Cod. 259/I,A-Wn 1799**,CDN-Hsmu M2149.L4,CDN-Mlr 111,CDN-Private-uhiz,CH-E 611,CH-Fco 2,"CH-ROM FiD 1, flyleaves (fragment)",CH-SG 545,CZ-HKm II A 4,CZ-OLu M IV 6,CZ-PLm 504 C 004,CZ-Pn II C 7,CZ-Pn XII A 24,CZ-Pn XV A 10,CZ-Pu I D 20,CZ-Pu XIV A 19,CZ-Pu XVII E 1,CZ-Pu XXIII A 2,D-KA Aug. LX,D-KNd 1161,F-Pn lat. 12044,F-Pn lat. 15181,F-Pnm Lat.11411 ff.37-38 (fragment),"F-Pnm NAL 2444, ff.53-55 (fragment)",F-Pnm NAL 2482 ff. 7-16 (fragment),I-Ac 693,I-Pc C55,I-Rvat SP B.79,NL-Uu 406,PL-WRu R 503,US-Cai 1911.142b,US-HA Rauner Codex MS 003203,USA-Phpc D-0zgch (fragment)
cantus_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001084,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
001546,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
001546:cs,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
001737,0,2,0,0,0,0,3,0,0,0,1,0,1,0,1,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,2,0,0,0,0
001737:cs,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
002000,0,1,1,1,0,0,1,0,0,0,1,0,1,0,1,1,1,0,1,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0
002000:cs,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
002642,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
002922:cs,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
sources = pd.Series(["A-Wn 1799**", 
"D-KNd 1161", 
"CDN-Hsmu M2149.L4", 
"D-KA Aug. LX", 
"F-Pn lat. 15181", 
"A-VOR Cod. 259/I", 
"CH-E 611", 
"CZ-PLm 504 C 004", 
"CZ-Pu XVII E 1", 
"F-Pn lat. 12044", 
"CZ-HKm II A 4", 
"CZ-Pn XV A 10", 
"CZ-Pu I D 20", 
"NL-Uu 406"]
)
sources

0           A-Wn 1799**
1            D-KNd 1161
2     CDN-Hsmu M2149.L4
3          D-KA Aug. LX
4       F-Pn lat. 15181
5      A-VOR Cod. 259/I
6              CH-E 611
7      CZ-PLm 504 C 004
8        CZ-Pu XVII E 1
9       F-Pn lat. 12044
10        CZ-HKm II A 4
11        CZ-Pn XV A 10
12         CZ-Pu I D 20
13            NL-Uu 406
dtype: object

In [15]:
canti = pd.Series(["605019", 
"001737", 
"002000", 
"003511", 
"004195", 
"007040a"])
canti

0     605019
1     001737
2     002000
3     003511
4     004195
5    007040a
dtype: object

In [16]:
# picking only the sources in our source list of interest
judjer1_src = judjer1[judjer1['siglum'].isin(sources)]
orisic_src = orisic[orisic['siglum'].isin(sources)]
consest_src = consest[consest['siglum'].isin(sources)]
judjer2_src = judjer2[judjer2['siglum'].isin(sources)]
bethnon_src = bethnon[bethnon['siglum'].isin(sources)]
cumesset_src = cumesset[cumesset['siglum'].isin(sources)]

In [17]:
# write just the fasta column of interest to a fasta file
with open('../data/judjer1_src.fasta', 'w') as f:
    for i in judjer1_src.index:
        f.write('> ' + judjer1_src.loc[i,'siglum'] + '\n')
        f.write(judjer1_src.loc[i,'volpiano'] + '\n')

with open('../data/orisic_src.fasta', 'w') as f:
    for i in orisic_src.index:
        f.write('> ' + orisic_src.loc[i,'siglum'] + '\n')
        f.write(orisic_src.loc[i,'volpiano'] + '\n')

with open('../data/consest_src.fasta', 'w') as f:
    for i in consest_src.index:
        f.write('> ' + consest_src.loc[i,'siglum'] + '\n')
        f.write(consest_src.loc[i,'volpiano'] + '\n')

with open('../data/judjer2_src.fasta', 'w') as f:
    for i in judjer2_src.index:
        f.write('> ' + judjer2_src.loc[i,'siglum'] + '\n')
        f.write(judjer2_src.loc[i,'volpiano'] + '\n')

with open('../data/bethnon_src.fasta', 'w') as f:
    for i in bethnon_src.index:
        f.write('> ' + bethnon_src.loc[i,'siglum'] + '\n')
        f.write(bethnon_src.loc[i,'volpiano'] + '\n')

with open('../data/cumesset_src.fasta', 'w') as f:
    for i in cumesset_src.index:
        f.write('> ' + cumesset_src.loc[i,'siglum'] + '\n')
        f.write(cumesset_src.loc[i,'volpiano'] + '\n')

In [18]:
# run mafft on the _src fastas
os.system('mafft --text --textmatrix 00_textmatrix_complete --globalpair --maxiterate 10000 ../data/judjer1_src.fasta > ../data/judjer1_src.aligned.fasta')
os.system('mafft --text --textmatrix 00_textmatrix_complete --globalpair --maxiterate 10000 ../data/orisic_src.fasta > ../data/orisic_src.aligned.fasta')
os.system('mafft --text --textmatrix 00_textmatrix_complete --globalpair --maxiterate 10000 ../data/consest_src.fasta > ../data/consest_src.aligned.fasta')
os.system('mafft --text --textmatrix 00_textmatrix_complete --globalpair --maxiterate 10000 ../data/judjer2_src.fasta > ../data/judjer2_src.aligned.fasta')
os.system('mafft --text --textmatrix 00_textmatrix_complete --globalpair --maxiterate 10000 ../data/bethnon_src.fasta > ../data/bethnon_src.aligned.fasta')
os.system('mafft --text --textmatrix 00_textmatrix_complete --globalpair --maxiterate 10000 ../data/cumesset_src.fasta > ../data/cumesset_src.aligned.fasta')

outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
nalphabets = 256
Score(9=0x39,9=0x39)=19.000000
Score(a=0x61,a=0x61)=19.000000
Score(b=0x62,b=0x62)=19.000000
Score(y=0x79,y=0x79)=19.000000
Score(Y=0x59,Y=0x59)=19.000000
Score(c=0x63,c=0x63)=19.000000
Score(d=0x64,d=0x64)=19.000000
Score(e=0x65,e=0x65)=19.000000
Score(f=0x66,f=0x66)=19.000000
Score(g=0x67,g=0x67)=19.000000
Score(h=0x68,h=0x68)=19.000000
Score(j=0x6a,j=0x6a)=19.000000
Score(i=0x69,i=0x69)=19.000000
Score(I=0x49,I=0x49)=19.000000
Score(k=0x6b,k=0x6b)=19.000000
Score(l=0x6c,l=0x6c)=19.000000
Score(m=0x6d,m=0x6d)=19.000000
Score(n=0x6e,n=0x6e)=19.000000
Score(o=0x6f,o=0x6f)=19.000000
Score(p=0x70,p=0x70)=19.000000
Score(q=0x71,q=0x71)=19.000000
Score(z=0x7a,z=0x7a)=19.000000
Score(Z=0x5a,Z=0x5a)=19.000000
Score(r=0x72,r=0x72)=19.000000
Score(s=0x73,s=0x73)=19.000000
Score(9=0x39,)=0x29)=19.000000
Score(a=0x61,A=0x41)=19.000000
Score(b=0x62,B=0x42)=19.000000
Score(y=0x79,Y=0x59)=19.000000
Score(c=0x63,C=0x43)=19

0

In [19]:
# convert from fasta to nexus using phyx, replace protein by standard and remove the intermediate nexus
os.system('for i in `ls ../data/*_src.aligned.fasta`; do pxs2nex -s $i -o $i.prenexus; done; rm phyx.logfile')
os.system('for i in `ls ../data/*prenexus`; do sed \'s/PROTEIN/STANDARD/g\' $i > $i.nexus; rm $i; done')

0

In [20]:
# concatenate using phyx
os.system('pxcat -s `ls ../data/*.nexus` -o ../data/concatenated.fasta -p ../data/partitions.txt; rm phyx.logfile')

0

In [21]:
# translate back to nexus and change the protein data type to standard
os.system('pxs2nex -s ../data/concatenated.fasta -o ../data/concatenated.prenexus; rm phyx.logfile')
os.system('sed \'s/PROTEIN/STANDARD/g\' ../data/concatenated.prenexus > ../data/concatenated.nexus; rm ../data/concatenated.prenexus')

0

In [22]:
# removed manually spaces and special characters in the source names that are not taken by mrbayes_volpiano
#A-VOR_Cod_259_I
#CH-E_611
#CZ-HKm_II_A_4
#CZ-PLm_504_C_004
#CZ-Pn_XV_A_10
#CZ-Pu_I_D_20
#D-KA_Aug_LX
#F-Pn_lat_12044
#NL-Uu_406
#A-Wn_1799
#CDN-Hsmu_M2149_L4
#CZ-Pu_XVII_E_1
#D-KNd_1161
#F-Pn_lat_15181
#NL-Uu_406
#F-Pn_lat_12044
#F-Pn_lat_15181

In [23]:
# note, removed manually the duplicates on concatenated/concatenated.nexus
# >' A-VOR Cod. 259/I'
# cdefed-dgfghgfed-efedecegfghgedefeedeghkhhgfgg-hgfefgg--fedefg-fed--fghgfgghgedefeedeghkhhhghkgffefgfedefeddefgfg-edefeehghkg--------e-fd-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# >' CH-E 611'
# cdfffdcd----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# >' CH-E 611'
# cffffedd----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# >' NL-Uu 406'
# cdfffe-dgfg-------------------------eghhgh------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------gggf-def---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ff-----------------------------------------cdfeggf-----------------------------
# >' NL-Uu 406'
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------gggf-defghhgg-------------hkkkk-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# >' F-Pn lat. 12044'
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ggfdfgg-hgfgh----ghi-jh--------------------------------------------------------
# >' F-Pn lat. 15181'
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ggfdfgg-hgfghghjhfhk-jgh-gfgh-gfgfedff---dfcdf-ggf-hk-hgfghhgg-----kkjk-hgghgkk


In [24]:
judjer1.loc[:,['siglum', 'idx']]

,siglum,idx
6,A-VOR Cod. 259/I,6
12,A-Wn 1799**,12
17,CDN-Hsmu M2149.L4,17
28,CH-E 611,28
30,CH-Fco 2,30
44,CZ-HKm II A 4,44
48,CZ-OLu M IV 6,48
55,CZ-PLm 504 C 004,55
69,CZ-Pn II C 7,69
82,CZ-Pn XV A 10,82


In [25]:
orisic.loc[:,['siglum', 'idx']]

,siglum,idx
8,A-VOR Cod. 259/I,8
13,A-Wn 1799**,13
18,CDN-Hsmu M2149.L4,18
20,CDN-Mlr 111,20
29,CH-E 611,29
31,CH-Fco 2,31
46,CZ-HKm II A 4,46
47,CZ-OLu M IV 6,47
50,CZ-PLm 504 C 004,50
67,CZ-Pn II C 7,67


In [26]:
consest.loc[:,['siglum', 'idx']]

,siglum,idx
0,A-KN 1010,0
3,A-VOR Cod. 259/I,3
9,A-Wn 1799**,9
14,CDN-Hsmu M2149.L4,14
25,CH-E 611,25
41,CZ-HKm II A 4,41
52,CZ-PLm 504 C 004,52
59,CZ-Pn II C 7,59
79,CZ-Pn XV A 10,79
86,CZ-Pu I D 20,86


In [27]:
judjer2.loc[:,['siglum', 'idx']]

,siglum,idx
7,A-VOR Cod. 259/I,7
11,A-Wn 1799**,11
16,CDN-Hsmu M2149.L4,16
27,CH-E 611,27
35,CH-SG 545,35
45,CZ-HKm II A 4,45
56,CZ-PLm 504 C 004,56
68,CZ-Pn II C 7,68
83,CZ-Pn XV A 10,83
92,CZ-Pu I D 20,92


In [28]:
bethnon.loc[:,['siglum', 'idx']]

,siglum,idx
1,A-VOR Cod. 259/I,1
2,A-VOR Cod. 259/I,2
22,CH-E 611,22
23,CH-E 611,23
24,CH-E 611,24
40,CZ-HKm II A 4,40
51,CZ-PLm 504 C 004,51
57,CZ-Pn II C 7,57
70,CZ-Pn XII A 24,70
78,CZ-Pn XV A 10,78


In [29]:
cumesset.loc[:,['siglum', 'idx']]

,siglum,idx
4,A-VOR Cod. 259/I,4
10,A-Wn 1799**,10
15,CDN-Hsmu M2149.L4,15
26,CH-E 611,26
42,CZ-HKm II A 4,42
53,CZ-PLm 504 C 004,53
63,CZ-Pn II C 7,63
71,CZ-Pn XII A 24,71
80,CZ-Pn XV A 10,80
87,CZ-Pu I D 20,87
